In [1]:
def loadGraphFromFile(file_name):

        n_nodes = 0
        n_edges = 0
        n_terminals = 0

        edges = []
        terminals = []

        with open(file_name, 'r') as file:

            for line in file:
                word_list = line.split()

                if(len(word_list) == 0):
                    continue

                if(word_list[0] == 'Nodes'):
                    n_nodes = int(word_list[1])

                if(word_list[0] == 'Edges'):
                    n_edges = int(word_list[1])

                if(word_list[0] == 'Terminals'):
                    n_terminals = int(word_list[1])

                if(word_list[0] == 'E'):
                    edges.append([int(word_list[1]) - 1, int(word_list[2]) - 1, int(word_list[3])])

                if(word_list[0] == 'T'):
                    terminals.append(int(word_list[1]) - 1)
                    
        return [n_nodes, n_edges, n_terminals, edges, terminals]
    
class SteinerTree:
    
    def __init__(self, file_name, opt):

        initList = loadGraphFromFile(file_name)

        self.name = file_name[-7:-4]
        self.opt = opt
        self.numNodes = initList[0]
        self.numEdges = initList[1]
        self.numTerms = initList[2]
        self.edges = initList[3]
        self.terminals = initList[4]
        
bGraphList = [
    ('b01.stp', 82),
    ('b02.stp', 83),
    ('b03.stp', 138),
    ('b04.stp', 59),
    ('b05.stp', 61),
    ('b06.stp', 122),
    ('b07.stp', 111),
    ('b08.stp', 104),
    ('b09.stp', 220),
    ('b10.stp', 86),
    ('b11.stp', 88),
    ('b12.stp', 174),
    ('b13.stp', 165),
    ('b14.stp', 235),
    ('b15.stp', 318),
    ('b16.stp', 127),
    ('b17.stp', 131),
    ('b18.stp', 218)
]

import os

bDataPath = os.path.abspath(os.getcwd()) + '\\..\\data\\B'

print(bDataPath)
filenames = os.listdir(bDataPath)
print(filenames)

bGraphs = []

for st_args in bGraphList:
    graphpath = bDataPath + '\\' + st_args[0]
    graphopt = st_args[1]
    bGraphs.append(SteinerTree(graphpath, graphopt))
    
print(len(bGraphs))
print(bGraphs[9].name)

G:\RI\Projekat\RI-Min-Steiner-Tree\code\..\data\B
['b01.stp', 'b02.stp', 'b03.stp', 'b04.stp', 'b05.stp', 'b06.stp', 'b07.stp', 'b08.stp', 'b09.stp', 'b10.stp', 'b11.stp', 'b12.stp', 'b13.stp', 'b14.stp', 'b15.stp', 'b16.stp', 'b17.stp', 'b18.stp', 'small_graph.stp']
18
b10


In [2]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse.csgraph import connected_components
import itertools
import numpy as np

In [3]:
#evaluira se svaki resenje, racunanjem vrednosti minimalnog razapinjuceg stabla
def evaluateSolution(p, solution_len, terminal_list, adj_matrix):
    
    total_value = np.sum(adj_matrix) / 2
    
    N = len(adj_matrix)
        
    s = 0

    selected_nodes = np.copy(terminal_list)

    for t in range(len(terminal_list)):

        if(selected_nodes[t] == 1):
            t = t+1
        else:
            selected_nodes[t] = p[s]
            s = s+1

    #pravi se nova matrica susedstva za izabrane cvorove
    M = sum(selected_nodes)

    new_adj_matrix = np.zeros((int(M), int(M)))

    m = 0

    for i in range(N):

        if(selected_nodes[i] == 1):

            n = 0

            for j in range(N):

                if(selected_nodes[j] == 1):

                    new_adj_matrix[m][n] = adj_matrix[i][j]

                    n = n+1
            m = m+1


    X = csr_matrix(new_adj_matrix.tolist())

    #provera da li je dobijen graf povezan
    #mora biti povezan da bi se racunao za Stainerovo drvo
    n_components, labels = connected_components(csgraph=X, directed=False, return_labels=True)
    
    if(n_components != 1):
        return (total_value, 0)
    
    #vraca gornje-trougaonu matricu povezanosti minimalnog razapinjuceg stabla
    Tcsr = minimum_spanning_tree(X)
    value = sum(sum(Tcsr.toarray().astype(int)))

    return (value, 1)

In [4]:
def bitfield(n, req_len):
    bf = np.array([int(digit) for digit in bin(n)[2:]])
    
    diff_len = req_len - bf.size
    
    return np.append(np.zeros(diff_len), bf)
    
    

def BFA(st: SteinerTree):
    
    #verovatnoca da ce mutirati jedna jedinka populacije
    mutation_chance = 0.2
    
    N = st.numNodes
    edge_number = st.numEdges
    term_number = st.numTerms
    optimal = st.opt
    edges = st.edges
    terminals = st.terminals
    
    terminal_list = np.zeros(N)
    
    for t in terminals:
        terminal_list[t] = 1
    
    #zaustavlja se iteriranje ako dobijemo optimalno resenje
    success = 0
    
    adj_matrix = np.zeros((N, N))
    
    for e in edges:
        adj_matrix[e[0]][e[1]] = e[2]
        adj_matrix[e[1]][e[0]] = e[2]
        
    total_value = np.sum(adj_matrix) / 2
        
    solution_len = N - term_number
    
    print(solution_len)
    
    #node_state = [0, 1]
    #all_trees = [list(i) for i in itertools.product(node_state, repeat=solution_len)]
    
    #print(all_trees[:4])
    
    best_value = total_value
    best_solution = []
    
    current_solution = []
    
    iteration = 0
    
    while iteration < pow(2, solution_len):
        
        tree = bitfield(iteration, solution_len)
            
        [tree_value, is_feasible] = evaluateSolution(tree, solution_len, terminal_list, adj_matrix)
        
        if is_feasible and best_value > tree_value:
            best_value = tree_value
            best_solution = np.copy(tree)
            
        if(iteration % 10000 == 0):
            print(best_value)
            
        if(iteration > 100000):
            return (best_solution, best_value, iteration)
        
        iteration = iteration + 1
            
    return (best_solution, best_value, iteration)

In [5]:
import os

small_graph = os.path.abspath(os.getcwd()) + '\\..\\data\\B' + '\\small_graph.stp'

small_graph_st = SteinerTree(small_graph, 12)

print(small_graph)

print(BFA(small_graph_st))

G:\RI\Projekat\RI-Min-Steiner-Tree\code\..\data\B\small_graph.stp
1
12
(array([0.]), 12, 2)


In [6]:
print(BFA(bGraphs[0]))

41
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
([], 359.0, 100001)
